In [51]:
import json

def extract_cves_from_misp(file_path):
    """Extracts CVEs and groups them by their event IDs from the MISP JSON file"""
    with open(file_path, "r") as f:
        misp_data = json.load(f)

    event_cve_mapping = {}

    for entry in misp_data:
        event_id = entry.get("event_id", "Unknown Event")
        cve_id = entry.get("name", "Unknown CVE")

        if event_id not in event_cve_mapping:
            event_cve_mapping[event_id] = []
        
        event_cve_mapping[event_id].append(cve_id)

    return event_cve_mapping

# ✅ Load the JSON file and extract CVEs grouped by events
misp_file_path = "/home/musharaf-misp-admin/musharaf/all_misp_cves.json"  # Path to the uploaded file
misp_cve_mapping = extract_cves_from_misp(misp_file_path)

# ✅ Print extracted mappings
print("Extracted MISP CVEs grouped by Event ID:")
for event, cves in misp_cve_mapping.items():
    print(f"\n🔹 Event ID: {event}")
    for cve in set(cves):  # Using set() to remove duplicates
        print(f"   - CVE: {cve}")


Extracted MISP CVEs grouped by Event ID:

🔹 Event ID: 80
   - CVE: CVE-2015-1641

🔹 Event ID: 84
   - CVE: CVE-2015-1770
   - CVE: CVE-2012-1856
   - CVE: CVE-2015-1641
   - CVE: CVE-2012-0158

🔹 Event ID: 91
   - CVE: CVE-2014-6332

🔹 Event ID: 92
   - CVE: CVE-2015-2545

🔹 Event ID: 106
   - CVE: CVE-2015-2546
   - CVE: CVE-2013-7331
   - CVE: CVE-2015-2545

🔹 Event ID: 113
   - CVE: CVE-2015-2545

🔹 Event ID: 122
   - CVE: CVE-2015-2545

🔹 Event ID: 129
   - CVE: CVE-2015-2545

🔹 Event ID: 222
   - CVE: CVE-2012-1258
   - CVE: CVE-2014-1761

🔹 Event ID: 230
   - CVE: CVE-2010-0738
   - CVE: CVE-2011-3544

🔹 Event ID: 231
   - CVE: CVE-2014-0515
   - CVE: CVE-2012-0507
   - CVE: CVE-2013-2551
   - CVE: CVE-2010-0188
   - CVE: CVE-2013-7331
   - CVE: CVE-2013-0074

🔹 Event ID: 236
   - CVE: CVE-2012-0158

🔹 Event ID: 245
   - CVE: CVE-2015-5122
   - CVE: CVE-2015-5119

🔹 Event ID: 274
   - CVE: CVE-2014-1761
   - CVE: CVE-2012-0158

🔹 Event ID: 278
   - CVE: CVE-2014-1761
   - CVE: CV

In [62]:
import xml.etree.ElementTree as ET

def extract_cves_cpes_from_nessus(file_path):
    """Extracts CVEs and CPEs from a Nessus scan report and groups by host"""
    tree = ET.parse(file_path)
    root = tree.getroot()

    nessus_cve_cpe = {}

    for report_host in root.findall(".//ReportHost"):
        host_name = report_host.get("name", "Unknown Host")
        
        for report_item in report_host.findall(".//ReportItem"):
            # Extract CVEs
            cve_list = report_item.find("cve")
            cve_ids = cve_list.text.split(",") if cve_list is not None else []

            # Extract CPE
            cpe_element = report_item.find("cpe")
            cpe = cpe_element.text if cpe_element is not None else "Unknown CPE"

            for cve_id in cve_ids:
                if host_name not in nessus_cve_cpe:
                    nessus_cve_cpe[host_name] = {}
                if cve_id not in nessus_cve_cpe[host_name]:
                    nessus_cve_cpe[host_name][cve_id] = set()

                if cpe != "Unknown CPE":
                    nessus_cve_cpe[host_name][cve_id].add(cpe)

    return nessus_cve_cpe

# ✅ Load Nessus file and extract CVEs & CPEs grouped by host
nessus_file_path = "/home/musharaf-misp-admin/musharaf/misp scan_f20myj.nessus"  # Update path accordingly
nessus_cve_cpe_mapping = extract_cves_cpes_from_nessus(nessus_file_path)

# ✅ Print extracted Nessus CVEs and CPEs grouped by hosts
print("\n🚀 Extracted Nessus CVEs and CPEs:")
if nessus_cve_cpe_mapping:
    for host, cve_data in nessus_cve_cpe_mapping.items():
        print(f"\n🏠 Host: {host}")
        for cve, cpes in cve_data.items():
            print(f"   🔹 {cve}")
            for cpe in cpes:
                print(f"      - CPE: {cpe}")
else:
    print("❌ No CVEs found in the Nessus scan report!")



🚀 Extracted Nessus CVEs and CPEs:

🏠 Host: 172.31.101.67
   🔹 CVE-2025-23083
      - CPE: cpe:/a:nodejs:node.js
   🔹 CVE-2024-28863
      - CPE: cpe:/a:node-tar_project:node-tar
   🔹 CVE-2016-2183
   🔹 CVE-2009-3129
      - CPE: cpe:/o:canonical:ubuntu_linux:22.04:-:lts
cpe:/o:canonical:ubuntu_linux:23.10
cpe:/o:canonical:ubuntu_linux:24.04:-:lts
p-cpe:/a:canonical:ubuntu_linux:libcjson-dev
p-cpe:/a:canonical:ubuntu_linux:libcjson1
   🔹 CVE-2024-29018
      - CPE: cpe:/o:canonical:ubuntu_linux:18.04:-:lts
cpe:/o:canonical:ubuntu_linux:24.04:-:lts
p-cpe:/a:canonical:ubuntu_linux:docker.io


In [61]:
def correlate_misp_nessus(misp_data, nessus_data):
    """Correlates CVEs from MISP (Threat Intel) and Nessus (Vulnerability Scan)"""
    correlation_results = {}
    common_cves = set()

    # Extracting CVEs from MISP (Flattening event-wise mapping)
    misp_cve_list = set()
    for event_cves in misp_data.values():
        misp_cve_list.update(event_cves)

    # Extracting CVEs from Nessus (Flattening host-wise mapping)
    nessus_cve_list = set()
    nessus_cpe_map = {}  # To store CPEs per CVE

    for host, cve_data in nessus_data.items():
        for cve, cpes in cve_data.items():
            nessus_cve_list.add(cve)
            if cve not in nessus_cpe_map:
                nessus_cpe_map[cve] = set()
            nessus_cpe_map[cve].update(cpes)

    # Find common CVEs
    common_cves = misp_cve_list & nessus_cve_list

    if common_cves:
        for cve in common_cves:
            correlation_results[cve] = {
                "misp_events": [event for event, cves in misp_data.items() if cve in cves],
                "nessus_hosts": [host for host, cve_data in nessus_data.items() if cve in cve_data],
                "nessus_cpes": list(nessus_cpe_map.get(cve, set()))  # Get correlated CPEs
            }

    return correlation_results, misp_cve_list, nessus_cve_list, common_cves

# ✅ Perform correlation
correlation_results, misp_cves, nessus_cves, common_cves = correlate_misp_nessus(misp_cve_mapping, nessus_cve_cpe_mapping)

# ✅ Print correlation results
print("\n🚀 Correlation Between MISP Threat Intel & Nessus Scan:")

if common_cves:
    for cve, data in correlation_results.items():
        print(f"\n🔹 CVE: {cve}")
        print("   - 🔥 MISP Events Containing This CVE:", data["misp_events"])
        print("   - 🏠 Nessus Hosts Affected:", data["nessus_hosts"])
        print("   - 🔎 CPEs Found in Nessus:", data["nessus_cpes"] if data["nessus_cpes"] else "None")
else:
    print("\n❌ No Matching CVEs Found Between MISP and Nessus!")
    print("\n🔎 MISP CVEs:", sorted(misp_cves))
    print("\n🔎 Nessus CVEs:", sorted(nessus_cves))



🚀 Correlation Between MISP Threat Intel & Nessus Scan:

🔹 CVE: CVE-2009-3129
   - 🔥 MISP Events Containing This CVE: ['294']
   - 🏠 Nessus Hosts Affected: ['172.31.101.67']
   - 🔎 CPEs Found in Nessus: ['cpe:/o:canonical:ubuntu_linux:22.04:-:lts\ncpe:/o:canonical:ubuntu_linux:23.10\ncpe:/o:canonical:ubuntu_linux:24.04:-:lts\np-cpe:/a:canonical:ubuntu_linux:libcjson-dev\np-cpe:/a:canonical:ubuntu_linux:libcjson1']
